In [2]:
%load_ext autoreload

In [12]:
%autoreload 2

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
import RNA 
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *

LANDSCAPE_TYPES ={"RNA": [0, 1], 'TF':['SIX6_REF_R1']} # for testing 
mutation_rate = 0.5
RNA_alphabet = "UCGA" 
DNA_alphabet = "TCGA"
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object=next(landscape_generator)
wt = landscape_object['starting_seqs']['startRNAL14_3']
landscape_oracle = landscape_object['landscape_oracle']

training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

Using TensorFlow backend.


B1L14RNA1 loaded



../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.










/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## DQN Explorer

In [14]:
from explorers.dqn_explorer import DQN_Explorer 

num_of_rounds = 5

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(train_epochs=10)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.14554841726711917
round: 1, cost: 101, evals: 883, top: 0.5794474621391882
round: 2, cost: 201, evals: 1674, top: 0.7085186640203461
round: 3, cost: 301, evals: 2423, top: 0.741473042412728
round: 4, cost: 401, evals: 2965, top: 0.741473042412728


In [15]:
landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object = next(landscape_generator)
wt = landscape_object['starting_seqs']['TF0']
landscape_oracle = landscape_object['landscape_oracle']

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=DNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(alphabet=DNA_alphabet, train_epochs=10, virtual_screen=20)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

1 TF landscapes loaded.


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.6781429101554447
round: 1, cost: 1, evals: 0, top: 0.6781429101554447
round: 2, cost: 1, evals: 0, top: 0.6781429101554447
round: 3, cost: 1, evals: 0, top: 0.6781429101554447
round: 4, cost: 1, evals: 0, top: 0.6781429101554447


In [3]:
from explorers.dqn_explorer import DQN_Explorer 

dqn_explorer_prod = DQN_Explorer(train_epochs=10)
dqn_explorer_prod.debug = False

In [ ]:
# run all evaluations 
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=3)

In [4]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.consistency_robustness_independence, num_starts=3)

loading landscapes RNA: [0, 1], TF:['SIX6_REF_R1']
B1L14RNA1 loaded
B1L50RNA1 loaded


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


1 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 1, evals: 0, top: 0.5437468768175177
round: 2, cost: 1, evals: 0, top: 0.5437468768175177
round: 3, cost: 1, evals: 0, top: 0.5437468768175177
round: 4, cost: 1, evals: 0, top: 0.5437468768175177
round: 5, cost: 1, evals: 0, top: 0.5437468768175177
round: 6, cost: 1, evals: 0, top: 0.5437468768175177
round: 7, cost: 1, evals: 0, top: 0.5437468768175177
round: 8, cost: 1, evals: 0, top: 0.5437468768175177
round: 9, cost: 1, evals: 0, top: 0.5437468768175177
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 1, evals: 0, top: 0.5437468768175177
round: 2, cost: 1, evals: 0, top: 0.5437468768175177
round: 3, cost: 1, evals: 0, top: 0.5437468768175177
round: 4, cost: 1, evals: 0, top: 0.5437468768175177
round: 5, cost: 1, evals: 0, top: 0.5437468768175177
round: 6, cost: 1, evals: 0, top: 0.5437468768175177
round: 7, cos

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not w

round: 0, cost: 1, evals: 0.0, top: 0.5437468768175177
round: 1, cost: 101, evals: 861.0, top: 0.6288789773482205
round: 2, cost: 201, evals: 1635.0, top: 0.6288789773482205
round: 3, cost: 301, evals: 2260.0, top: 0.6288789773482205
round: 4, cost: 401, evals: 2784.0, top: 0.6288789773482205
round: 5, cost: 499, evals: 3061.0, top: 0.6371175981360863
round: 6, cost: 599, evals: 3373.0, top: 0.7085186640203461
round: 7, cost: 697, evals: 3658.0, top: 0.7085186640203461
round: 8, cost: 796, evals: 3954.0, top: 0.7085186640203461
round: 9, cost: 896, evals: 4243.0, top: 0.7085186640203461


...

round: 0, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 1, cost: 101, evals: 735.0, top: 0.8870655176679174
round: 2, cost: 192, evals: 1426.0, top: 0.8870655176679174
round: 3, cost: 273, evals: 2036.0, top: 0.9366367047375582
round: 4, cost: 347, evals: 2518.0, top: 0.9366367047375582
round: 5, cost: 418, evals: 2848.0, top: 0.9366367047375582
round: 6, cost: 488, evals: 3100.0, top: 0.9366367047375582
round: 7, cost: 546, evals: 3395.0, top: 0.9366367047375582
round: 8, cost: 617, evals: 3674.0, top: 0.9366367047375582
round: 9, cost: 675, evals: 3883.0, top: 0.9366367047375582


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not w

round: 0, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 1, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 2, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 3, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 4, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 5, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 6, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 7, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 8, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 9, cost: 1, evals: 0.0, top: 0.33058796849874245


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not w

round: 0, cost: 1, evals: 0.0, top: 0.33058796849874245
round: 1, cost: 101, evals: 730.0, top: 0.9428421226240052
round: 2, cost: 199, evals: 1303.0, top: 0.9428421226240052
round: 3, cost: 290, evals: 1735.0, top: 0.9835071949861872
round: 4, cost: 381, evals: 2033.0, top: 0.9835071949861872
round: 5, cost: 471, evals: 2201.0, top: 0.9835071949861872
round: 6, cost: 548, evals: 2353.0, top: 0.9835071949861872
round: 7, cost: 625, evals: 2473.0, top: 0.9835071949861872
round: 8, cost: 693, evals: 2621.0, top: 0.9835071949861872
round: 9, cost: 761, evals: 2749.0, top: 0.9835071949861872
round: 0, cost: 1, evals: 0, top: 0.33058796849874245
round: 1, cost: 1, evals: 0, top: 0.33058796849874245
round: 2, cost: 1, evals: 0, top: 0.33058796849874245
round: 3, cost: 1, evals: 0, top: 0.33058796849874245
round: 4, cost: 1, evals: 0, top: 0.33058796849874245
round: 5, cost: 1, evals: 0, top: 0.33058796849874245
round: 6, cost: 1, evals: 0, top: 0.33058796849874245
round: 7, cost: 1, evals: 0

## BO Explorer

In [ ]:
%autoreload 2
from explorers.bo_explorer import BO_Explorer

num_of_rounds = 5
arch1 = NLNN(len(wt), hidden_dims=10, epochs=20)
arch2 = NLNN(len(wt), hidden_dims=20, epochs=20)
arch3 = NLNN(len(wt), hidden_dims=30, epochs=20)
arch4 = NLNN(len(wt), hidden_dims=40, epochs=20)
# ensemble we will use for BO Explorer
nn_model_1 = NN_model(landscape_oracle, arch1)
nn_model_2 = NN_model(landscape_oracle, arch2)
nn_model_3 = NN_model(landscape_oracle, arch3)
nn_model_4 = NN_model(landscape_oracle, arch4)
models = [nn_model_1, nn_model_2, nn_model_3, nn_model_4]
ensemble_model = Ensemble_models(models)
ensemble_model.update_model(training_sequences)

In [ ]:
# evaluate BO Explorer 
bo_explorer = BO_Explorer()
bo_explorer.debug = True
bo_explorer.set_model(ensemble_model)
bo_explorer.run(num_of_rounds, verbose=True)

In [22]:
from explorers.bo_explorer import BO_Explorer

bo_explorer_prod = BO_Explorer()
bo_explorer_prod.debug = False

In [ ]:
# run all evaluations 
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.scalability, num_starts=3)

In [ ]:
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.consistency_robustness_independence, num_starts=3)

loading landscapes RNA: [0, 1], TF:['SIX6_REF_R1']


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


B1L14RNA1 loaded
B1L50RNA1 loaded
1 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 101, evals: 10099, top: 0.55747785908442
round: 2, cost: 201, evals: 20199, top: 0.5712088413513224
round: 3, cost: 301, evals: 30299, top: 0.5712088413513224
round: 4, cost: 401, evals: 40399, top: 0.6124017881520295
round: 5, cost: 500, evals: 50498, top: 0.6535947349527367
round: 6, cost: 690, evals: 60598, top: 0.6535947349527367
round: 7, cost: 789, evals: 70697, top: 0.6535947349527367
round: 8, cost: 889, evals: 80797, top: 0.6535947349527367
round: 9, cost: 989, evals: 90897, top: 0.6563409418820252
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 101, evals: 10100, top: 0.6728181310782162
round: 2, cost: 201, evals: 20200, top: 0.6728181310782162
round: 3, cost: 301, evals: 30300, top: 0.6728181310782162
round: 4, cost: 400, evals: 40399, top: 0.6728181310782162
round: 5,

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not w

round: 0, cost: 1, evals: 0.0, top: 0.5437468768175177
round: 1, cost: 101, evals: 4549.0, top: 0.5437468768175177
round: 2, cost: 201, evals: 7705.0, top: 0.5794474621391882
round: 3, cost: 301, evals: 12742.0, top: 0.6563409418820252
round: 4, cost: 401, evals: 22218.0, top: 0.7085186640203461
round: 5, cost: 501, evals: 27096.0, top: 0.8018893853389146
round: 6, cost: 601, evals: 36601.0, top: 0.8018893853389146
round: 7, cost: 701, evals: 46138.0, top: 0.8018893853389146
round: 8, cost: 801, evals: 55670.0, top: 0.8018893853389146
round: 9, cost: 901, evals: 65146.0, top: 0.8018893853389146


...

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not w

round: 0, cost: 1, evals: 0.0, top: 0.6443675592872286
round: 1, cost: 101, evals: 9913.0, top: 0.6443675592872286
round: 2, cost: 201, evals: 19832.0, top: 0.6443675592872286
round: 3, cost: 301, evals: 29739.0, top: 0.6443675592872286
round: 4, cost: 401, evals: 39644.0, top: 0.6443675592872286
round: 5, cost: 501, evals: 49563.0, top: 0.6443675592872286
round: 6, cost: 601, evals: 59499.0, top: 0.6443675592872286
round: 7, cost: 701, evals: 69417.0, top: 0.6443675592872286
round: 8, cost: 801, evals: 79336.0, top: 0.6443675592872286
round: 9, cost: 901, evals: 89259.0, top: 0.6443675592872286
round: 0, cost: 1, evals: 0, top: 0.6443675592872286
round: 1, cost: 101, evals: 10100, top: 0.6443675592872286
round: 2, cost: 201, evals: 20200, top: 0.6443675592872286
round: 3, cost: 301, evals: 30300, top: 0.6443675592872286
round: 4, cost: 401, evals: 40400, top: 0.6443675592872286
round: 5, cost: 501, evals: 50500, top: 0.6443675592872286
round: 6, cost: 601, evals: 60600, top: 0.6443675

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not w

round: 0, cost: 1, evals: 0.0, top: 0.9027297098882465
round: 1, cost: 101, evals: 9911.0, top: 0.9027297098882465
round: 2, cost: 201, evals: 19834.0, top: 0.9027297098882465
round: 3, cost: 301, evals: 29746.0, top: 0.9027297098882465
round: 4, cost: 401, evals: 39659.0, top: 0.9027297098882465
round: 5, cost: 501, evals: 49570.0, top: 0.9027297098882465
round: 6, cost: 601, evals: 59485.0, top: 0.9027297098882465
round: 7, cost: 701, evals: 69414.0, top: 0.9027297098882465
round: 8, cost: 801, evals: 79318.0, top: 0.9027297098882465
round: 9, cost: 901, evals: 89221.0, top: 0.9027297098882465


/home/alexander/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.103182). Check your callbacks.
  % delta_t_median)
